<a href="https://colab.research.google.com/github/anindabitm/zindi_yield_prediction/blob/main/Pytorch_yield_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall fastai -y -q
!pip install fastai -q

     |████████████████████████████████| 194kB 14.9MB/s 
     |████████████████████████████████| 61kB 10.3MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os

Mounted at /content/drive


In [ ]:
import pandas as pd
import gc
import numpy as np
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler,LabelEncoder
import glob
import re
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import backend as K
from keras.layers import Dense,Conv2D,TimeDistributed,Flatten,Input,AvgPool2D,LSTM,LeakyReLU,Concatenate,Dropout,Embedding
from keras import Model
from fastai.vision.all import *
import fastai

In [ ]:
print(fastai.__version__)

2.2.5


In [ ]:
data = pd.read_csv('/content/drive/My Drive/yield-estimate/Train.csv')
data['train'] = 1
test = pd.read_csv('/content/drive/My Drive/yield-estimate/test_field_ids_with_year.csv')
test['Quality'] = 3
test['train'] = 0

In [ ]:
addl_data = pd.read_csv('/content/drive/My Drive/yield-estimate/fields_w_additional_info.csv')

In [ ]:
soil = addl_data.iloc[:,:12]
soil.head()

,Field_ID,soil_bdod_5-15cm_mean,soil_cec_5-15cm_mean,soil_cfvo_5-15cm_mean,soil_clay_5-15cm_mean,soil_nitrogen_5-15cm_mean,soil_ocd_5-15cm_mean,soil_ocs_0-30cm_mean,soil_phh2o_5-15cm_mean,soil_sand_5-15cm_mean,soil_silt_5-15cm_mean,soil_soc_5-15cm_mean
0,MH2O0YH,118.0,257.0,85.0,463.0,2458.0,323.0,57.0,59.0,308.0,229.0,330.0
1,O9TURWL,120.0,288.0,84.0,462.0,2108.0,306.0,58.0,61.0,243.0,295.0,280.0
2,35AFSDD,120.0,263.0,102.0,495.0,1991.0,288.0,58.0,60.0,271.0,234.0,284.0
3,PM05EG9,122.0,206.0,66.0,431.0,2432.0,337.0,65.0,57.0,332.0,237.0,335.0
4,V7PZBCG,124.0,167.0,116.0,439.0,566.0,302.0,53.0,56.0,326.0,235.0,187.0


In [ ]:
soil['Quality'] = np.concatenate((data.Quality.values,test.Quality.values))
soil.head()

,Field_ID,soil_bdod_5-15cm_mean,soil_cec_5-15cm_mean,soil_cfvo_5-15cm_mean,soil_clay_5-15cm_mean,soil_nitrogen_5-15cm_mean,soil_ocd_5-15cm_mean,soil_ocs_0-30cm_mean,soil_phh2o_5-15cm_mean,soil_sand_5-15cm_mean,soil_silt_5-15cm_mean,soil_soc_5-15cm_mean,Quality
0,MH2O0YH,118.0,257.0,85.0,463.0,2458.0,323.0,57.0,59.0,308.0,229.0,330.0,3
1,O9TURWL,120.0,288.0,84.0,462.0,2108.0,306.0,58.0,61.0,243.0,295.0,280.0,2
2,35AFSDD,120.0,263.0,102.0,495.0,1991.0,288.0,58.0,60.0,271.0,234.0,284.0,3
3,PM05EG9,122.0,206.0,66.0,431.0,2432.0,337.0,65.0,57.0,332.0,237.0,335.0,2
4,V7PZBCG,124.0,167.0,116.0,439.0,566.0,302.0,53.0,56.0,326.0,235.0,187.0,2


In [ ]:
soil.to_csv('/content/drive/My Drive/yield-estimate/Soil.csv',index=False)

In [ ]:
data.drop(['Yield'],axis=1,inplace=True)
data.shape

(2977, 4)

In [ ]:
data = pd.concat([data,test]).reset_index(drop=True)
data.head()

,Field_ID,Year,Quality,train
0,MH2O0YH,2019,3,1
1,O9TURWL,2019,2,1
2,35AFSDD,2019,3,1
3,PM05EG9,2019,2,1
4,V7PZBCG,2019,2,1


In [ ]:
climate=pd.DataFrame()
climate['Field_ID'] = addl_data['Field_ID']
climate['Year'] = data['Year']
climate['Quality'] = data['Quality']

In [ ]:
cols = addl_data.columns[12:]
cols = list(cols[:168])
fea = []
for col in cols:
  fea.append(re.sub("_\d+_", "",col))

for idx in tqdm(range(len(climate))):
  year = climate.loc[idx,'Year']
  field = climate.loc[idx,'Field_ID']
  if (year == 2016):
    col = list(range(12,180))
  elif (year == 2017):
    col = list(range(180,348))
  elif (year == 2018):
    col = list(range(348,516))
  else:
   col = list(range(516,684))

  climate.loc[idx,fea] = addl_data.iloc[idx,col].values
  
climate.head()

,Field_ID,Year,Quality,climate1_aet,climate1_def,climate1_pdsi,climate1_pet,climate1_pr,climate1_ro,climate1_soil,climate1_srad,climate1_swe,climate1_tmmn,climate1_tmmx,climate1_vap,climate1_vpd,climate1_vs,climate2_aet,climate2_def,climate2_pdsi,climate2_pet,climate2_pr,climate2_ro,climate2_soil,climate2_srad,climate2_swe,climate2_tmmn,climate2_tmmx,climate2_vap,climate2_vpd,climate2_vs,climate3_aet,climate3_def,climate3_pdsi,climate3_pet,climate3_pr,climate3_ro,climate3_soil,climate3_srad,climate3_swe,...,climate10_pdsi,climate10_pet,climate10_pr,climate10_ro,climate10_soil,climate10_srad,climate10_swe,climate10_tmmn,climate10_tmmx,climate10_vap,climate10_vpd,climate10_vs,climate11_aet,climate11_def,climate11_pdsi,climate11_pet,climate11_pr,climate11_ro,climate11_soil,climate11_srad,climate11_swe,climate11_tmmn,climate11_tmmx,climate11_vap,climate11_vpd,climate11_vs,climate12_aet,climate12_def,climate12_pdsi,climate12_pet,climate12_pr,climate12_ro,climate12_soil,climate12_srad,climate12_swe,climate12_tmmn,climate12_tmmx,climate12_vap,climate12_vpd,climate12_vs
0,MH2O0YH,2019,3,119.0,1405.0,370.0,1524.0,4.0,0.0,287.0,2543.0,0.0,112.0,285.0,1246.0,138.0,260.0,122.0,1367.0,260.0,1489.0,7.0,0.0,236.0,2609.0,0.0,116.0,297.0,1166.0,161.0,260.0,170.0,1464.0,120.0,1634.0,14.0,1.0,200.0,2585.0,0.0,...,-600.0,1191.0,96.0,5.0,99.0,2057.0,0.0,109.0,269.0,1637.0,80.0,250.0,791.0,357.0,-570.0,1148.0,83.0,4.0,93.0,2029.0,0.0,112.0,271.0,1615.0,85.0,240.0,1067.0,0.0,-400.0,1067.0,125.0,6.0,215.0,1880.0,0.0,100.0,267.0,1680.0,69.0,210.0
1,O9TURWL,2019,2,115.0,1376.0,410.0,1491.0,4.0,0.0,287.0,2581.0,0.0,108.0,288.0,1260.0,138.0,220.0,119.0,1362.0,290.0,1481.0,7.0,0.0,237.0,2639.0,0.0,118.0,305.0,1157.0,173.0,230.0,178.0,1518.0,160.0,1696.0,15.0,1.0,203.0,2652.0,0.0,...,-580.0,1228.0,79.0,4.0,102.0,2095.0,0.0,109.0,279.0,1640.0,90.0,230.0,743.0,413.0,-540.0,1156.0,78.0,4.0,96.0,2050.0,0.0,111.0,280.0,1628.0,93.0,200.0,1078.0,0.0,-360.0,1078.0,126.0,6.0,215.0,1883.0,0.0,99.0,273.0,1682.0,75.0,190.0
2,35AFSDD,2019,3,111.0,1401.0,410.0,1512.0,4.0,0.0,278.0,2571.0,0.0,109.0,288.0,1276.0,137.0,240.0,112.0,1384.0,290.0,1496.0,7.0,0.0,230.0,2629.0,0.0,119.0,306.0,1165.0,174.0,240.0,173.0,1548.0,160.0,1721.0,15.0,1.0,197.0,2632.0,0.0,...,-600.0,1240.0,73.0,4.0,100.0,2095.0,0.0,110.0,280.0,1644.0,91.0,240.0,718.0,435.0,-560.0,1153.0,75.0,4.0,93.0,2041.0,0.0,112.0,280.0,1637.0,93.0,200.0,1084.0,0.0,-360.0,1084.0,135.0,7.0,289.0,1875.0,0.0,100.0,274.0,1690.0,76.0,200.0
3,PM05EG9,2019,2,123.0,1378.0,370.0,1501.0,3.0,0.0,299.0,2550.0,0.0,110.0,282.0,1236.0,134.0,250.0,128.0,1325.0,250.0,1453.0,8.0,0.0,244.0,2612.0,0.0,115.0,295.0,1171.0,158.0,240.0,169.0,1475.0,120.0,1644.0,14.0,1.0,207.0,2592.0,0.0,...,-580.0,1161.0,106.0,5.0,102.0,2062.0,0.0,108.0,264.0,1648.0,73.0,240.0,826.0,297.0,-550.0,1123.0,86.0,4.0,95.0,2041.0,0.0,110.0,268.0,1619.0,81.0,220.0,1053.0,0.0,-390.0,1053.0,121.0,6.0,189.0,1898.0,0.0,98.0,263.0,1688.0,64.0,210.0
4,V7PZBCG,2019,2,643.0,988.0,270.0,1631.0,20.0,1.0,427.0,2438.0,0.0,161.0,320.0,1575.0,173.0,270.0,294.0,1370.0,140.0,1664.0,18.0,1.0,300.0,2576.0,0.0,174.0,332.0,1478.0,208.0,290.0,395.0,1409.0,-280.0,1804.0,35.0,2.0,234.0,2641.0,0.0,...,-590.0,1250.0,237.0,12.0,1179.0,2057.0,0.0,150.0,292.0,1998.0,89.0,240.0,1213.0,34.0,-550.0,1247.0,122.0,6.0,1123.0,2106.0,0.0,152.0,296.0,1960.0,99.0,250.0,1199.0,0.0,-380.0,1199.0,194.0,66.0,1214.0,1930.0,0.0,143.0,291.0,1972.0,87.0,270.0


In [ ]:
climate.to_csv('/content/drive/My Drive/yield-estimate/Climate.csv',index=False)

In [ ]:
climate.iloc[:,3:].values.reshape(4032,12,14)[:,0,:]

array([[ 119., 1405.,  370., ..., 1246.,  138.,  260.],
       [ 115., 1376.,  410., ..., 1260.,  138.,  220.],
       [ 111., 1401.,  410., ..., 1276.,  137.,  240.],
       ...,
       [1043.,  265.,  432., ..., 1931.,  105.,  289.],
       [1131.,  187.,  404., ..., 1850.,  111.,  273.],
       [1121.,  205.,  409., ..., 1851.,  111.,  277.]])

# Creating folds

In [ ]:
from sklearn.model_selection import train_test_split,KFold
kf = KFold(n_splits=5,shuffle=True,random_state=2020)
fold_df = train.copy(deep=True)
crop_yield = np.zeros(shape=(1055,1))
for fold,(train_index, test_index) in enumerate(kf.split(train.Field_ID.values,train.Yield.values)):
  fold_df.loc[test_index,f'fold'] = fold+1

fold_df.head()

In [ ]:
fold_df.to_csv('/content/drive/My Drive/yield-estimate/folds.csv',index=False)

# Data reading section

In [ ]:
# Copying in the zip files from Google Drive (alternately just upload them)
!cp '/content/drive/My Drive/yield-estimate/image_arrays_train.zip' .
!cp '/content/drive/My Drive/yield-estimate/image_arrays_test.zip' .

In [ ]:
# Extract
!unzip -q image_arrays_train.zip
!unzip -q image_arrays_test.zip

In [ ]:
!ls

drive		   image_arrays_test.zip  image_arrays_train.zip
image_arrays_test  image_arrays_train	  sample_data


In [ ]:
train_fns = glob.glob('./image_arrays_train/*.*')
test_fns = glob.glob('./image_arrays_test/*.*')
print('The number of training files are:',len(train_fns),' and number of test filenames are:',len(test_fns))

The number of training files are: 2977  and number of test filenames are: 1055


In [ ]:
train = pd.read_csv('/content/drive/My Drive/yield-estimate/Train.csv')
print(train.shape)
train.head()

(2977, 4)


,Field_ID,Year,Quality,Yield
0,MH2O0YH,2019,3,3.686
1,O9TURWL,2019,2,5.657
2,35AFSDD,2019,3,3.082
3,PM05EG9,2019,2,2.707
4,V7PZBCG,2019,2,2.679


In [ ]:
ss = pd.read_csv('/content/drive/My Drive/yield-estimate/SampleSubmission.csv')
print(ss.shape)
ss.head()

(1055, 2)


,Field_ID,Yield
0,E9UZCEA,0
1,1WGGS1Q,0
2,EG2KXE2,0
3,HC3GQXF,0
4,7AK6GFK,0


In [ ]:
train = train.loc[(train['Quality']==3) | (train['Quality']==2),:].reset_index(drop=True)
train.shape

(2552, 4)

In [ ]:
train.head()

,Field_ID,Year,Quality,Yield
0,MH2O0YH,2019,3,3.686
1,O9TURWL,2019,2,5.657
2,35AFSDD,2019,3,3.082
3,PM05EG9,2019,2,2.707
4,V7PZBCG,2019,2,2.679


In [ ]:
def get_array(df,length,train_status):
  array = []
  for idx in tqdm(range(length)):
    if train_status == 1:
      fn = './image_arrays_train/' + df.loc[idx,'Field_ID'] + '.npy'

    else:
      fn = './image_arrays_test/' + df.loc[idx,'Field_ID'] + '.npy'
    
    img = np.load(fn)[:,:32,:32]
    img = img.transpose((1,2,0))
    img = img.reshape(32,32,30,-1)
    img = img.transpose((3,0,1,2))
    img = img/np.max(img)
    img = img.astype('float16')
    array.append(img)
  return np.stack(array)

In [ ]:
X = get_array(train,len(train),1)

In [ ]:
X.shape

(2552, 12, 32, 32, 30)

In [ ]:
np.save('/content/drive/My Drive/yield-estimate/train',X)

In [ ]:
test = get_array(ss,len(ss),0)
print(test.shape)
np.save('/content/drive/My Drive/yield-estimate/test',test)


(1055, 12, 32, 32, 30)


# Modelling

In [ ]:
#train = pd.read_csv('/content/drive/My Drive/yield-estimate/Train.csv')
#train = train.loc[(train['Quality']==3) | (train['Quality']==2),:].reset_index(drop=True)
train = pd.read_csv('/content/drive/My Drive/yield-estimate/folds.csv')
train.shape

(2552, 5)

In [ ]:
ss = pd.read_csv('/content/drive/My Drive/yield-estimate/SampleSubmission.csv')
print(ss.shape)

(1055, 2)


In [ ]:
climate_df = pd.read_csv('/content/drive/My Drive/yield-estimate/Climate.csv')
climate_train = climate_df.iloc[:2977]
climate_train = climate_train.loc[(climate_train['Quality']==3) | (climate_train['Quality']==2),:].reset_index(drop=True)
climate_train.fillna(value=0,inplace=True)
climate_train.shape

(2552, 171)

In [ ]:
soil = pd.read_csv('/content/drive/My Drive/yield-estimate/Soil.csv')
soil_train = soil.iloc[:2977]
soil_train = soil_train.loc[(soil_train['Quality']==3) | (soil_train['Quality']==2),:].reset_index(drop=True)
soil_train.fillna(value=0,inplace=True)
soil_train.shape

(2552, 13)

In [ ]:
test_year = pd.read_csv('/content/drive/My Drive/yield-estimate/test_field_ids_with_year.csv')
test_year.head()

,Field_ID,Year
0,E9UZCEA,2019
1,1WGGS1Q,2019
2,EG2KXE2,2019
3,HC3GQXF,2019
4,7AK6GFK,2019


In [ ]:
X1 = np.load('/content/drive/My Drive/yield-estimate/train.npy')

In [ ]:
X1 = X1.transpose(0,1,4,2,3)
X1.shape

(2552, 12, 30, 32, 32)

In [ ]:
X1 = X1[:,:6,:,:,:]
X1.shape

(2552, 6, 30, 32, 32)

In [ ]:
X2 = climate_train.iloc[:,3:].values
y = train.Yield.values
X2.shape,y.shape

((2552, 168), (2552,))

In [ ]:
scaler1 = StandardScaler()
X2 = scaler1.fit_transform(X2)
X2 = X2.reshape(-1,12,14)
X2.shape

(2552, 12, 14)

In [ ]:
X3 = soil_train.iloc[:,1:12].values
scaler2 = StandardScaler()
X3 = scaler2.fit_transform(X3)
X3.shape

(2552, 11)

In [ ]:
X4 = train.Year.values
le = LabelEncoder()
X4 = le.fit_transform(X4)
X4.shape

(2552,)

In [ ]:
test1 = np.load('/content/drive/My Drive/yield-estimate/test.npy')
test1.shape

(1055, 12, 32, 32, 30)

In [ ]:
test1 = test1.transpose(0,1,4,2,3)
dummy = np.zeros(shape=(1055,1))
test1.shape,dummy.shape

((1055, 12, 30, 32, 32), (1055, 1))

In [ ]:
test1 = test1[:,:6,:,:,:]
test1.shape

(1055, 6, 30, 32, 32)

In [ ]:
climate_test = climate_df.iloc[2977:]
climate_test.fillna(value=0,inplace=True)
climate_test.shape

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


(1055, 171)

In [ ]:
soil_test = soil.iloc[2977:]
soil_test.fillna(value=0,inplace=True)
soil_test.shape

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


(1055, 13)

In [ ]:
test2 = climate_test.iloc[:,3:].values
test2 = scaler1.transform(test2)
test2 = test2.reshape(-1,12,14)
test2.shape

(1055, 12, 14)

In [ ]:
test3 = soil_test.iloc[:,1:12].values
test3 = scaler2.transform(test3)
test3.shape

(1055, 11)

In [ ]:
test4 = test_year.Year.values
test4 = le.transform(test4)
test4.shape

(1055,)

In [ ]:
class YieldDataset():
  def __init__(self,x1,x2,x3,y):
    self.x1 = x1
    self.x2 = x2
    self.x3 = x3
    self.y = y
    

  def __len__(self):
    return(len(self.x1))

  def __getitem__(self,idx):
    x1 = self.x1[idx]
    x2 = self.x2[idx]
    x3 = self.x3[idx]
    y = self.y[idx]
    return torch.tensor(x1,dtype=torch.float),torch.tensor(x2,dtype=torch.float),torch.tensor(x3,dtype=torch.float),torch.tensor(y,dtype=torch.float)

In [ ]:
class YieldNet(nn.Module):
  def __init__(self):
    super(YieldNet, self).__init__()
    self.conv1 = ConvLayer(30, 64)
    self.conv2 = ConvLayer(64, 128)
    self.pool = AdaptiveConcatPool2d(size=None)
    self.fc1 = nn.Linear(1536,512)
    self.fc2 = nn.Linear(691,1)

  def forward(self, x1,x2,x3):
    batch_size, time_steps, C, H, W = x1.size()
    x1 = x1.view(batch_size*time_steps,C,H,W)
    x1 = F.relu(self.conv1(x1))
    x1 = F.relu(self.conv2(x1))
    x1 = self.pool(x1)
    x1 = x1.view(batch_size,-1)
    inp1 = F.relu(self.fc1(x1))
    x2 = x2.view(batch_size,-1)
    inp = torch.cat([inp1,x2,x3],dim=1)
    out = self.fc2(inp)
    return out

model = YieldNet()

In [ ]:
#tdl = learn.dls.test_dl(test1)
#batch_size, time_steps, C, H, W = tdl.one_batch().size()

gc.collect()

756

In [ ]:
train_ds = YieldDataset(x_train1,y_train)
valid_ds = YieldDataset(x_valid1,y_valid)

dls = DataLoaders.from_dsets(train_ds,valid_ds)
x,y = first(dls.train)
model(x).shape

torch.Size([64, 1])

In [ ]:
((100663296/64)/128)

12288.0

In [ ]:
##########################FOLDS ##################################################
test_ds = YieldDataset(test1,test2,test3,dummy)
final_preds = np.zeros(shape=(test1.shape[0],1))

for f in range(0,5):
  print('#'*10,'In fold:',f+1,'#'*10)
  model = YieldNet()
  train_index = list(train.loc[train['fold']!=f+1].index)
  test_index = list(train.loc[train['fold']==f+1].index)
  x_train1,x_valid1 = X1[train_index],X1[test_index]
  x_train2,x_valid2 = X2[train_index],X2[test_index]
  x_train3,x_valid3 = X3[train_index],X3[test_index]
  x_train4,x_valid4 = X4[train_index],X4[test_index]
  y_train,y_valid = y[train_index],y[test_index]
  print(x_train1.shape,x_train2.shape,x_train3.shape,x_train4.shape,x_valid1.shape,x_valid2.shape,x_valid3.shape,x_valid4.shape)

  train_ds = YieldDataset(x_train1,x_train2,x_train3,y_train)
  valid_ds = YieldDataset(x_valid1,x_valid2,x_valid3,y_valid)

  dls = DataLoaders.from_dsets(train_ds,valid_ds)
  dls = dls.cuda()

  learn = Learner(dls,model,loss_func=MSELossFlat(),metrics=[rmse]).to_fp16()

  learn.fit_one_cycle(15,slice(3e-03,3e-02),cbs=SaveModelCallback())

  tdl = learn.dls.test_dl(test_ds)
  
  preds,_ = learn.get_preds(dl=tdl)

  final_preds += preds.numpy()

  print('#'*10,'Finished predictions!!','#'*10)
  print('')
  del dls
  del model
  del learn
  gc.collect()
  


########## In fold: 1 ##########
(2041, 6, 30, 32, 32) (2041, 12, 14) (2041, 11) (2041,) (511, 6, 30, 32, 32) (511, 12, 14) (511, 11) (511,)


epoch,train_loss,valid_loss,_rmse,time
0,8.194236,5.499785,2.345162,00:04
1,4.646342,2.920510,1.708950,00:03
2,3.526086,3.801250,1.949679,00:03
3,4.884637,5.427475,2.329694,00:03
4,4.720910,3.549597,1.884037,00:03
5,4.204156,3.426302,1.851027,00:03
6,3.483257,2.862901,1.692011,00:03
7,3.117975,2.819451,1.679122,00:03
8,2.772807,3.043460,1.744551,00:03
9,2.571760,2.887970,1.699403,00:03


Better model found at epoch 0 with valid_loss value: 5.49978494644165.
Better model found at epoch 1 with valid_loss value: 2.9205102920532227.
Better model found at epoch 6 with valid_loss value: 2.862901449203491.
Better model found at epoch 7 with valid_loss value: 2.819451332092285.
Better model found at epoch 10 with valid_loss value: 2.8190884590148926.
Better model found at epoch 12 with valid_loss value: 2.800905466079712.
Better model found at epoch 14 with valid_loss value: 2.800053119659424.


########## Finished predictions!! ##########

########## In fold: 2 ##########
(2041, 6, 30, 32, 32) (2041, 12, 14) (2041, 11) (2041,) (511, 6, 30, 32, 32) (511, 12, 14) (511, 11) (511,)


epoch,train_loss,valid_loss,_rmse,time
0,8.285322,6.009381,2.451404,00:03
1,4.798836,2.606873,1.614581,00:03
2,3.998492,5.225610,2.285959,00:03
3,6.700023,16.812494,4.100304,00:03
4,8.704836,3.720021,1.928735,00:03
5,6.343195,4.068006,2.016930,00:03
6,5.046350,2.747918,1.657685,00:03
7,3.946624,3.626049,1.904219,00:03
8,3.383161,2.723641,1.650346,00:03
9,3.120031,2.556173,1.598804,00:03


...

Better model found at epoch 0 with valid_loss value: 5.086925983428955.
Better model found at epoch 1 with valid_loss value: 2.8351967334747314.
Better model found at epoch 5 with valid_loss value: 2.707146406173706.
Better model found at epoch 7 with valid_loss value: 2.5260372161865234.
Better model found at epoch 9 with valid_loss value: 2.513528823852539.
Better model found at epoch 10 with valid_loss value: 2.4467549324035645.
Better model found at epoch 11 with valid_loss value: 2.4013304710388184.


########## Finished predictions!! ##########

########## In fold: 5 ##########
(2042, 6, 30, 32, 32) (2042, 12, 14) (2042, 11) (2042,) (510, 6, 30, 32, 32) (510, 12, 14) (510, 11) (510,)


epoch,train_loss,valid_loss,_rmse,time
0,9.300026,2.507362,1.583465,00:03
1,5.099607,2.522525,1.588246,00:03
2,4.765451,4.103572,2.025728,00:03
3,11.049525,5.319185,2.306336,00:03
4,8.568411,2.592291,1.610059,00:03
5,5.798234,2.289076,1.512969,00:03
6,4.363254,2.020132,1.421313,00:03
7,3.633945,2.233048,1.494339,00:03
8,3.136521,2.238905,1.496297,00:03
9,2.990381,2.338218,1.529123,00:03


Better model found at epoch 0 with valid_loss value: 2.5073623657226562.
Better model found at epoch 5 with valid_loss value: 2.289076089859009.
Better model found at epoch 6 with valid_loss value: 2.0201315879821777.


########## Finished predictions!! ##########



# Generating predictions

In [ ]:
ss['Yield'] = final_preds/5

In [ ]:
#ss['Yield'] = crop_yield/5
ss.head()

,Field_ID,Yield
0,E9UZCEA,3.037109
1,1WGGS1Q,3.356641
2,EG2KXE2,3.371094
3,HC3GQXF,3.393750
4,7AK6GFK,3.115625


In [ ]:
ss.to_csv('/content/drive/My Drive/yield-estimate/fastai_5fold4.csv',index=False)